### 데이터 설명

- 기간 :2018년 9월 9일 ~ 2020년 11월 8일
- 한 시간 간격의 사용자 행동 데이터
- columns : 사용자 수, 세션 수, 신규 방문자 수, 페이지 뷰 수

    - 세션수는 사이트의 모든 사용자가 시작한 개별 세션 수를 나타냅니다.사용자가 사이트를 방문한 후 30분 이상 어떤 작업도 수행하지 않은 경우 이후에 발생한 모든 작업은 새로운 세션으로 간주됩니다. 사용자가 사이트를 떠났다가 30분 내에 다시 방문하면 해당 방문은 원래 세션의 일부로 기록됩니다. 언제든지 최초 세션의 방문자는 새로운 세션 및 새로운 사용자로 간주됩니다. 이 기간에 동일한 사용자가 추가로 발생시키는 세션은 추가 세션으로 집계되지만, 추가 사용자로는 집계되지 않습니다.


In [24]:
import pandas as pd
import numpy as np
train = pd.read_csv('dacon_data/train.csv', encoding = 'euc-kr')
info_user = pd.read_csv('dacon_data/info_user.csv', encoding = 'euc-kr',index_col ='Unnamed: 0')

In [7]:
info_user[~info_user.id.isnull()]

,id,c_time,entered_competition_cnt,code_share_cnt,talk_board_cnt,country_code,python_level,ds_level
0.0,15994.0,2017-05-02 23:58,94.0,306.0,477.0,82.0,1.0,1.0
1.0,19195.0,2018-08-08 18:21,1.0,0.0,0.0,82.0,1.0,1.0
2.0,16339.0,2018-08-08 19:57,1.0,0.0,0.0,82.0,1.0,1.0
3.0,7290.0,2018-08-08 19:59,0.0,0.0,0.0,82.0,1.0,1.0
4.0,21287.0,2018-08-08 20:26,0.0,0.0,0.0,82.0,1.0,1.0
...,...,...,...,...,...,...,...,...
21687.0,11584.0,2020-12-08 22:53,1.0,0.0,0.0,82.0,1.0,1.0
21688.0,6337.0,2020-12-08 22:54,1.0,0.0,0.0,82.0,1.0,1.0
21689.0,2505.0,2020-12-08 23:07,1.0,0.0,0.0,82.0,1.0,1.0
21690.0,14182.0,2020-12-08 23:09,1.0,0.0,0.0,82.0,1.0,1.0


## 1.1 정형화된 데이터의 요소

1. 데이터의 종류
    1. 수치형 데이터 : 숫자를 이용해 표현할 수 있는 데이터
        1. 연속형 데이터 : 일정 범위 안에서 어떤 값이든 취할 수 있는 데이터   Ex, 풍속, 지속시간
        2. 이산데이터 : 횟수와 같은 정수값만 취할수있는 데이터                Ex, 사건의 발생빈도
    2. 범주형 데이터 : 가능한 범주 안의 값만을 취하는 데이터
        1. 이진데이터 : 범주형 데이터 중에서도 두 값중 하나를 갖는 아주 특수한 경우
        2. 순서형 데이터 : 범주안의 값들이 순위를 갖는 데이터                 ex, 평점

## 1.2. 테이블 데이터
- 피처 feature : 일반적으로 테이블의 열을 의미.    
- 결과 outcome : 실험이나 연구에서 나오는 결과를 의미
- 레코드 record : 테이블의 각 행

In [8]:
train

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18
...,...,...,...,...,...
19003,2020-11-08 19:00:00,124,123,19,3128
19004,2020-11-08 20:00:00,166,159,29,4864
19005,2020-11-08 21:00:00,184,173,32,3426
19006,2020-11-08 22:00:00,163,155,34,2845


## 1.3. 위치 추정
: **데이터를 살표보는 가장 기초적인 단계는 각 피처(변수)의 ‘대푯값’을 구하는 것이다.**

이는 곧 **대부분의 값이 어디쯤 위치 하는지 (중심경향성)을 나타내는 추정값**이다.

In [11]:
train['Date'] = train['DateTime'].map(lambda x : str(x)[:10])

train_18_0909 = train[train['Date'] == '2018-09-09']
train_20_1108 = train[train['Date'] == '2020-11-08']

In [15]:
train_18_0909.head()

,DateTime,사용자,세션,신규방문자,페이지뷰,Date
0,2018-09-09 00:00:00,19,19,8,206,2018-09-09
1,2018-09-09 01:00:00,20,19,9,259,2018-09-09
2,2018-09-09 02:00:00,12,9,1,48,2018-09-09
3,2018-09-09 03:00:00,10,10,2,102,2018-09-09
4,2018-09-09 04:00:00,6,5,3,18,2018-09-09


In [20]:
train_18_0909.shape[0]

24

### 1) 평균

In [18]:
print('train_18_0909_페이지 뷰 : ', train_18_0909['페이지뷰'].mean())
print('train_20_1108_페이지 뷰 : ', train_20_1108['페이지뷰'].mean())

train_18_0909_페이지 뷰 :  76.08333333333333
train_20_1108_페이지 뷰 :  2103.5833333333335


### 2) 절사 평균
값들을 크기 순으로 정렬하여, 정해진 개수의 극단값을 제외한 나머지 값들의 평균이다. 이러한 방식을 통해 평균에 비해 극단값의 영향을 덜 받는다.

In [23]:
from scipy.stats import trim_mean

print('train_18_0909_페이지 뷰 : ', trim_mean(train_18_0909['페이지뷰'], 0.1))
print('train_20_1108_페이지 뷰 : ',trim_mean(train_20_1108['페이지뷰'], 0.1))

train_18_0909_페이지 뷰 :  66.55
train_20_1108_페이지 뷰 :  2020.45


### 다른 방법 : np.quantile()

In [30]:
quantile_1 = np.quantile(train_18_0909['페이지뷰'], 0.1) # 0.1 값이 무엇은지 알 수 있음
quantile_9 =np.quantile(train_18_0909['페이지뷰'], 0.9)  # 0.1 값이 무엇은지 알 수 있음

In [33]:
train_18_0909[(train_18_0909['페이지뷰'] > quantile_1) & ( train_18_0909['페이지뷰'] < quantile_9)]['페이지뷰'].mean()

63.888888888888886

### 3) 가중 평균
: 각 데이터 값에  사용자가 지정한 가중치를 곱한 값의 총합(wx)을 가중치의 총합(w)으로 나눈값이다. 
- 이러한 평균을 사용하는 이유는 데이터를 수집할때, 서로 다른 대조군의 수가 다르기 때문에 이를 보정하는 방법으로 부족한 소수 그룹에 더 높은 가중치를 주는 것이다. 

In [35]:
np.average(train_18_0909['페이지뷰'], weights=train_18_0909['사용자'])

98.70106761565836

### 4) 중간값
: 데이터를 일렬로 정렬했을때, 한 가운데 위치하는 값이다. 이는 평균에 비해 극단값의 영향을 덜 받게 된다.

In [36]:
print('train_18_0909_페이지 뷰 : ', train_18_0909['페이지뷰'].median())
print('train_20_1108_페이지 뷰 : ', train_20_1108['페이지뷰'].median())

train_18_0909_페이지 뷰 :  53.0
train_20_1108_페이지 뷰 :  1932.0


<aside>
💡 결과를 잘못 이해하게 하는 특잇값(극단값)들의 영향을 덜 받은 것을 '로버스트하다' 라고 한다. 이 말은, 모델이나 데이터가 견고하다 라도 생각하면 된다. 즉, 조금더 로버스트한 대표값을 얻기 위해서 평균보다 중간값 혹은 절사 평균 등이 도움이 된다.

</aside>

## 1.4. 변이 추정

**: 변이 ( variability ) 는 데이터의 값이 얼마나 밀집 혹은 퍼져있는지를 나타내는 산포도이다.**

**: 중심경향도만으로는 집단에 대한 성격과 분포를 파악하는데 부족하므로, 측정된 데이터가 어떻게 분포하고 있는지에 대해 파악해야 데이터를 제대로 이해할 수 있다.**

: 변이 추정은 **편차**를 기본으로 하는데, 

- 편차란, 관측 데이터와 위치 추정값 사이의 차이이며, 이는 데이터가 중앙값을 주변으로 얼마나 퍼져있는가를 나타낸다.

### 1) 평균 절대 편차
편차의 절댓값의 평균을 구하는 것 

$$ \frac{\sum_{i=1}^{n} \left | x_{i} - \bar{x} \right|}{n}$$

In [45]:
def mean_absolute_deviation(data):
    data_mean = data['페이지뷰'].mean()
    n = data.shape[0]
    return sum(np.absolute(data['페이지뷰'] - data_mean)) / n

mean_absolute_deviation(train_18_0909)

53.02083333333335

### 2) 분산과 표준 편차
: 분산은 제곱편차의 평균이고, 표준편차는 분산의 제곱근이다. 
- 표준편차는 분산에서 제곱한 것을 다시 원래의 데이터의 척도(scale)로 돌아가게 해주기 때문에, 해석하기 쉽다.

$$분산 = \frac{\sum_{i=1}^{n} \left (  x_{i} - \bar{x}  \right )^{2}}{n-1} $$

In [47]:
print('분산 : ', train_18_0909['페이지뷰'].var())
print('표준편차 : ', train_18_0909['페이지뷰'].std())

분산 :  4429.210144927537
표준편차 :  66.55231134173731


![https://blog.kakaocdn.net/dn/AbXmU/btqMGeSAB8B/smYy1asxp3d31ssXKFuuoK/img.png](https://blog.kakaocdn.net/dn/AbXmU/btqMGeSAB8B/smYy1asxp3d31ssXKFuuoK/img.png)

왼 ) 분산이 큼  오 ) 분산이 작음

- 분산이 큼 : 값들이 퍼져있음. 평균이 값들의 특성을 잘 나타낸다고 말하기 어려움
- 분산이 작음 : 값들이 모여져있음. 값들이 평균값에 모아져 있음.

### 3) 중위 절대 편차 ( MAD : Median Absolute Deviation )
: 분간, 표준편차, 평균절대편차 모두 로버스트하지 않고 특잇값에 민간하기 때문에 이것을 사용한다.

이것은 관측값에서 중앙값을 빼고, 그 차이에 절댓값을 취한 값들의 중앙값을 구하는 것이다.

In [50]:
from statsmodels import robust

robust.scale.mad(train_18_0909['페이지뷰'])

51.14977653844326

> 통계학에서는 표준편차를 평균절대편차 보다 더 선호하는데, 그 이유는 수학적으로 제곱한 값이 절댓값보다 통계모델을 다룬데 더 편리하다는 통계 이론이 뒷받침하고 있어서 라고 한다.또한, 표준편차는 항상 평균절대편차보다 크다. 그리고 평균 절대편차는 중위절대편차보다 크다.